<a href="https://colab.research.google.com/github/RadhavaramHarika/Harika_INFO5731_Spring2020/blob/master/Harika_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as panda

def getDataFromUrl(url):
  with urllib.request.urlopen(url) as file:
    web_content = file.read()
    b_soup = BeautifulSoup(web_content)
  return b_soup

def getUserReviewList(urlData):
  review_data = urlData.find_all("div",{"class":"lister-item-content"})
  reviews_list = []
  for each in review_data:
    title = each.find('a').get_text()
    review = each.find(class_ = "content").find(class_ = "text show-more__control").get_text()
    reviews_list.append([title, review])
  return reviews_list

def getHundredUserReviewList(urlData):
  total_reviews = getUserReviewList(urlData)
  count=len(total_reviews)
  base_url = loadMoreReviews(urlData)
  while len(total_reviews)!=100:
    if base_url is not None:
      content = getDataFromUrl(base_url)
      for each in getUserReviewList(content):
        total_reviews.append(each)
    else:
      break
    count = len(total_reviews)
  return total_reviews

def extractFirstReview(reviewList):
  user_review = reviewList[0]
  return user_review

def retriewTitles(reviewList):
  titles = [each.find('a').get_text() for each in reviewList]
  return titles 

def retriewTotalReview(reviewList):
  totalReviews = [each.find(class_ = "content").find(class_ = "text show-more__control").get_text() for each in reviewList]
  return totalReviews 

def loadMoreReviews(url_data):
  moreReviews = url_data.select(".load-more-data") if url_data else None
  if len(moreReviews):
    ajax_url = moreReviews[0]["data-ajaxurl"]
    url = "https://www.imdb.com"+ajax_url+"?ref_=undefined&paginationKey="
    try:    
      key = moreReviews[0]["data-key"]
    except KeyError:
      key = None
    print("K",key)
  return url+key if key else None

def uploadReviewsToCSV(reviewList):
  csv_dict = {"Review Title":[each[0] for each in reviewList],
              "Total Review":[each[1] for each in reviewList]}
  datafr = panda.DataFrame(csv_dict)
  datafr.to_csv('UserReview.csv', index = False)
  return datafr


url = "https://www.imdb.com/title/tt7286456/reviews?start=0"
web_content = getDataFromUrl(url)

user_reviews = getHundredUserReviewList(web_content)
print(len(user_reviews))

first_review = extractFirstReview(user_reviews)
print(first_review)

uploadReviewsToCSV(user_reviews)

K g4wp7crnq4ytiyid7kxxrojwrprmsazhzfmxvlnomwklyczuf43o6ss6oiyvvmzndj4k4u5ja6rjnzuwxlyxe75bw2mfnzy
100
[" As a viewer that actually went to TIFF and witnessed this film and didn't want to believe the hype, it is an absolute MASTERPIECE and Phoenix is a certified legend.\n", "I was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another Joker based film. I thought this looked solid at best and even a bit too pretentious in the trailer, but in here to say I was incredibly wrong. This is a massive achievement of cinema that's extremely rare in a day and age of cgi nonsense and reboots. While this is somewhat of a reboot of sorts, the standalone origin tale is impeccable from start to finish and echoes resemblance to the best joker origin comics from the past. Joaquin bleeds, sweats, and cries his every drop into this magnificently dedicated performance. Heath Ledger would be proud. This is undoubtedly the greatest acting performance sin

,Review Title,Total Review
0,As a viewer that actually went to TIFF and wi...,I was a person that saw all the hype and claim...
1,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul..."
2,Only certain people can relate\n,This is a movie that only those who have felt ...
3,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o..."
4,The Hype is real\n,Most of the time movies are anticipated like t...
...,...,...
95,Astonishing Masterpiece\n,What an incredible ride this was. I was almost...
96,Nonsense plot\n,Arthur Fleck lives with his invalid mother. He...
97,I feel like everyone is brain washed\n,I thought this movie was complete and utter ga...
98,I made an account just to rate my disappoint ...,I was expecting a masterpiece and oscar worthy...


In [17]:
# Write your code here
import nltk
import csv,os,math,unicodedata,re
from google.colab import drive,files
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter 
from textblob import TextBlob
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as panda
from sklearn.feature_extraction.text import TfidfVectorizer
import string,io
from google.colab import drive,files

def remove_punctuations(samp_sent):
  removed_punct = []
  translator = str.maketrans('', '', string.punctuation) 
  for each in samp_sent:
    removed_punct.append(each.translate(translator))
  return removed_punct

def remove_numbers(samp_sent):
  removed_numb = []
  for each in samp_sent:
    removed_numb.append(re.sub(r"\d", "", each))
  return removed_numb

def remove_stop_words(samp_sent):
  stops = stopwords.words('english')
  no_stops = []
  for each in samp_sent:
    words = each.split()
    no_stops.append(' '.join([w for w in words
                              if w not in stops]))
  return no_stops

def lower_sentences(samp_sent):
  lowers = []
  for each in samp_sent:
    lowers.append(each.lower())
  return lowers

def stemming(samp_sent_tokens):
  stem_sent = []
  ps = PorterStemmer()
  for each in samp_sent_tokens:
    words = word_tokenize(each)
    stem_sent.append([ps.stem(w) for w in words])
  return stem_sent

def lemmatized_sent_tokens(samp_sent_tokens):
  lmtzr = WordNetLemmatizer() 
  lem_sent_tokens = []
  for each in samp_sent_tokens:
    lem_sent_tokens.append([lmtzr.lemmatize(w) for w in each])
  return lem_sent_tokens

def uploadPreprocessedDataToCSV(reviews, data):
  no_punct_sent = remove_punctuations(reviews)
  no_stop_sent = remove_stop_words(no_punct_sent)
  no_numb_sent = remove_numbers(no_stop_sent)
  lowered_sent = lower_sentences(no_numb_sent)
  stem_sent_tokens = stemming(lowered_sent)
  lemmatized_tokens = lemmatized_sent_tokens(stem_sent_tokens)
  #[word_tokenize(each) for each in lowered_sent]
  data["Preprocessed Reviews"] = lemmatized_tokens
  data.to_csv("UserReview.csv")
  uploading_directory(data)
  return data

def uploading_directory(data):
  drive.mount('/content/gdrive')
  filenames = os.listdir('/content/gdrive/My Drive/Colab Notebooks')
  print(filenames)

  os.chdir('/content/gdrive/My Drive/Colab Notebooks')
  text_dict = ['Review Title','Total Review','Preprocessed Reviews']
  for i in range(len(data)):
    text_dict.append([data['Review Title'][i],data['Total Review'][i],data['Preprocessed Reviews'][i]])

  with open('UserReviews.csv', 'w') as csvFile:
    csvWriter = csv.writer(csvFile)
    csvWriter.writerows(text_dict)
    csvFile.close()
  return
  
data = panda.read_csv('UserReview.csv')
uploadPreprocessedDataToCSV(data['Total Review'],data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['Untitled0.ipynb', 'My_first_in_Class_exercise.ipynb', 'My_First_in_class_Assignement_Harika.ipynb', 'Untitled1.ipynb', 'exercise_02_data_collection', 'Copy of In_class_exercise_02 (2).ipynb', 'Untitled2.ipynb', 'textContents.csv', 'Copy of In_class_exercise_02 (1).ipynb', 'Harika_In_class_exercise_02.ipynb', 'Assignment1_denton_housing.csv', 'Harika_INFO5731_Assignment_One.ipynb', 'Untitled', 'Untitled3.ipynb', 'Harika_In_class_exercise_03 (1).ipynb', 'Harika_In_class_exercise_03.ipynb', 'Legal_case.txt', 'stopwords'

,Review Title,Total Review,Preprocessed Reviews
0,As a viewer that actually went to TIFF and wi...,I was a person that saw all the hype and claim...,"[i, person, saw, hype, claim, masterpiec, over..."
1,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul...","[everi, movi, come, truli, make, impact, joaqu..."
2,Only certain people can relate\n,This is a movie that only those who have felt ...,"[thi, movi, felt, alon, isol, truli, relat, yo..."
3,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o...","[truli, masterpiec, the, best, hollywood, film..."
4,The Hype is real\n,Most of the time movies are anticipated like t...,"[most, time, movi, anticip, like, end, fall, s..."
...,...,...,...
95,Astonishing Masterpiece\n,What an incredible ride this was. I was almost...,"[what, incred, ride, i, almost, motionless, th..."
96,Nonsense plot\n,Arthur Fleck lives with his invalid mother. He...,"[arthur, fleck, live, invalid, mother, he, suf..."
97,I feel like everyone is brain washed\n,I thought this movie was complete and utter ga...,"[i, thought, movi, complet, utter, garbag, i, ..."
98,I made an account just to rate my disappoint ...,I was expecting a masterpiece and oscar worthy...,"[i, expect, masterpiec, oscar, worthi, film, h..."


In [18]:
import spacy
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
from nltk.chunk import ne_chunk
from nltk.tree import Tree
from nltk.chunk import conlltags2tree, tree2conlltags
from spacy.gold import GoldParse 
from spacy.language import EntityRecognizer


def posTagging(samp_sent):
  pos_tags = []
  print("\nQues 3.1")
  print("Nouns\tVerbs\tAdjectives\tAdverbs")
  for each in samp_sent:
    tags = nltk.pos_tag(' '.join(each))
    pos_tags.append(tags)
    num_N = ' '.join([w[1] for w in tags]).count('NN')
    num_V = ' '.join([w[1] for w in tags]).count('VBP')
    num_Adj = ' '.join([w[1] for w in tags]).count('JJ')
    num_Adv = ' '.join([w[1] for w in tags]).count('RB')
    print(str(num_N) +'\t'+str(num_V)+'\t'+str(num_Adj)+'\t\t'+str(num_Adv))
  return

def constituencyParsing(samp_Sent):
  pattern = 'NP: {<DT>?<JJ>*<NN>}'
  cp = nltk.chunk.regexp.RegexpParser(pattern)
  print("\nQues 3.2")
  print("Constituency Parsing Tree for all the sentences")
  for each in samp_Sent:
    cs = cp.parse(nltk.pos_tag(each))
    #print(cs)
    cs.pprint()
  return

def to_nltk_tree(node):
  if node.n_lefts + node.n_rights > 0:
      return Tree('-'.join([node.orth_,node.tag_]), [to_nltk_tree(child) for child in node.children])
  else:
      return '-'.join([node.orth_,node.tag_])

def dependencyParsingTree(samp_sent):
  print("\nQues 3.2")
  print("\nDependency Parsing Tree for all the sentences")
  nlp = spacy.load('en')
  for each in samp_sent:
    if len(each)>1:
      e_sent = nlp(' '.join(each))
      [to_nltk_tree(sent.root).pretty_print() for sent in e_sent.sents]
  return

def nameEntityRecog(samp_sent):
  print("\nQues 3.3")
  print("\nEntity Recognition")
  nlp = spacy.load('en_core_web_sm')
  for each in samp_sent:
    doc = nlp(' '.join(each))
    for entity in doc.ents:
      print(entity.text, entity.label_) 
  return
 
  '''nlp = spacy.load('en', entity = False, parser = False) 
  
  doc_list = [] 
  doc = nlp('Llamas make great pets.') 
  doc_list.append(doc) 
  gold_list = [] 
  gold_list.append(GoldParse(doc, [u'ANIMAL', u'O', u'O', u'O'])) 
    
  ner = EntityRecognizer(nlp.vocab, entity_types = ['ANIMAL']) 
  ner.update(doc_list, gold_list)'''

posTagging(data["Preprocessed Reviews"])
constituencyParsing(data["Preprocessed Reviews"])
dependencyParsingTree(data["Preprocessed Reviews"])
nameEntityRecog(data["Preprocessed Reviews"])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


ImportError: ignored